<a href="https://colab.research.google.com/github/dineshssdn-867/CSE541_Computer_Vision_2022_5/blob/master/code/Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and config files

In [1]:
# !pip install scipy==1.1.0

In [2]:
import platform
platform.python_version()

'3.6.10'

In [3]:
# !pip install tensorflow==2.1
# !pip install keras==2.3.1

In [4]:
import cv2
import scipy.io
import os
import gc
import sklearn.preprocessing
import matplotlib
import tensorflow
import keras.optimizers
import warnings
import scipy.io as sio
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import numpy as np
import networkx as nx
import pandas as pd
from matplotlib.animation import FuncAnimation, PillowWriter 
from keras import backend as K
from keras.models import Sequential
from keras.regularizers import l2
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D, AveragePooling3D 
from tensorflow.keras.layers import BatchNormalization, Activation, Input
from tensorflow.keras import layers
from PIL import Image
# from scipy.misc import imresize
from keras.models import model_from_json
from datetime import datetime
from keras.utils.data_utils import get_file
from keras.utils.layer_utils import get_source_inputs
from keras.utils import layer_utils
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [5]:
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import AveragePooling3D
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling3D

In [6]:
C3D_MEAN_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/c3d_mean.npy'

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

## Parameters

In [9]:
c3d_model_weights = 'E:/LY Project/C3D_Sport1M_weights_keras_2.2.4.h5'
abnormal_videos_path="E:/LY Project/Anomaly Videos"
normal_videos_path="E:/LY Project/Training-Normal-Videos-Part-2"
raw_normal_train_features_c3d="E:/LY Project/Raw Normal videos features C3d"
raw_abnormal_train_features_c3d="E:/LY Project/Raw Anomaly videos features C3d"
processed_normal_train_features_c3d="E:/LY Project/Pre-processed Normal Video features C3d"
processed_abnormal_train_features_c3d="E:/LY Project/Pre-processed Anomaly video features C3d"
raw_normal_train_features_i3d="E:/LY Project/Raw Normal videos features i3d"
raw_abnormal_train_features_i3d="E:/LY Project/Raw Anomaly videos features i3d"
processed_normal_train_features_i3d="E:/LY Project/Pre-processed Normal Video features i3d"
processed_abnormal_train_features_i3d="E:/LY Project/Pre-processed Anomaly video features i3d"
output_dir = "E:/LY Project/Trained_models/"
output_folder_c3d = 'E:/LY Project/output_videos'
output_folder_i3d = 'E:/LY Project/output_videos_i3d'

In [11]:
frame_height = 240
frame_width = 320
channels = 3

c3d_frame_count = 16


In [12]:
features_per_bag = 32

## Chunks work

In [13]:
def divide_chunks(l, n):
    """Apply divide_chunks to an array, getting chunks of
    of specified size
    """
    for i in range(0, len(l), n): 
        yield l[i:i + n]

## Array Util

In [14]:
def sliding_window(arr, size, stride):
    """Apply sliding window to an array, getting chunks of
    of specified size using the specified stride
    :param arr: Array to be divided
    :param size: Size of the chunks
    :param stride: Number of frames to skip for the next chunk
    :returns: Tensor with the resulting chunks
    :rtype: np.ndarray
    """
    num_chunks = int((len(arr) - size) / stride) + 2
    result = []
    for i in range(0,  num_chunks * stride, stride):
        if len(arr[i:i + size]) > 0:
            result.append(arr[i:i + size])
    return np.array(result)


def interpolate(features, features_per_bag):
    """Transform a bag with an arbitrary number of features into a bag
    with a fixed amount, using interpolation of consecutive features
    :param features: Bag of features to pad
    :param features_per_bag: Number of features to obtain
    :returns: Interpolated features
    :rtype: np.ndarray
    """
#     feature_size = np.array(features).shape[0]
    feature_size = np.array(features).shape[1]
    interpolated_features = np.zeros((features_per_bag, feature_size))
    interpolation_indices = np.round(np.linspace(0, len(features) - 1, num=features_per_bag + 1))
    count = 0
    for index in range(0, len(interpolation_indices)-1):
        start = int(interpolation_indices[index])
        end = int(interpolation_indices[index + 1])

        assert end >= start

        if start == end:
            temp_vect = features[start, :]
        else:
            temp_vect = np.mean(features[start:end+1, :], axis=0)

        temp_vect = temp_vect / np.linalg.norm(temp_vect)

        if np.linalg.norm(temp_vect) == 0:
            print("Error")

        interpolated_features[count,:]=temp_vect
        count = count + 1
    del interpolation_indices
    gc.collect(generation=2)
    return np.array(interpolated_features)


def extrapolate(outputs, num_frames):
    """Expand output to match the video length
    :param outputs: Array of predicted outputs
    :param num_frames: Expected size of the output array
    :returns: Array of output size
    :rtype: np.ndarray
    """

    extrapolated_outputs = []
    extrapolation_indices = np.round(np.linspace(0, len(outputs) - 1, num=num_frames))
    for index in extrapolation_indices:
        extrapolated_outputs.append(outputs[int(index)])
    del extrapolation_indices
    gc.collect(generation=2)
    return np.array(extrapolated_outputs)

# Video Util

In [15]:
def get_video_clips(video_path, frame_count):
    """Divides the input video into non-overlapping clips
    :param video_path: Path to the video
    :returns: Array with the fragments of video
    :rtype: np.ndarray
    """
    frames = get_video_frames(video_path)
    clips = sliding_window(frames, frame_count, frame_count)
    return clips, len(frames)

In [16]:
def get_video_frames(video_path):
    """Reads the video given a file path
    :param video_path: Path to the video
    :returns: Video as an array of frames
    :rtype: np.ndarray
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            del frame
        else:
            break
    cap.release()
    del cap
    gc.collect(generation=2)
    return frames

# Video pre-processing C3d

---



In [17]:
def preprocess_input_c3d(video):
    """Preprocess video input to make it suitable for feature extraction.
    The video is resized, cropped, resampled and training mean is substracted
    to make it suitable for the network
    :param video: Video to be processed
    :returns: Preprocessed video
    :rtype: np.ndarray
    """

    intervals = np.ceil(np.linspace(0, video.shape[0] - 1, 16)).astype(int)
    frames = video[intervals]

    # Reshape to 128x171
    reshape_frames = np.zeros((frames.shape[0], 128, 171, frames.shape[3]))
    for i, img in enumerate(frames):
#         heatmap = imresize(heatmap, input_dims, interp='bicubic', mode='F')
# heatmap = np.array(Image.fromarray(obj=heatmap, mode='F').resize(size=input_dims, resample=Image.BICUBIC))
# img = np.array(Image.fromarray(obj=img).resize(size=(128, 171), resample =Image.BICUBIC))
#         img = imresize(img, (128, 171), 'bicubic')
        img = np.array(Image.fromarray(obj=img).resize(size=(171, 128), resample =Image.BICUBIC))
        reshape_frames[i, :, :, :] = img
        del img

    mean_path = get_file('c3d_mean.npy',
                         C3D_MEAN_PATH,
                         cache_subdir='models',
                         md5_hash='08a07d9761e76097985124d9e8b2fe34')

    mean = np.load(mean_path)
    reshape_frames -= mean
    # Crop to 112x112
    reshape_frames = reshape_frames[:, 8:120, 30:142, :]
    # Add extra dimension for samples
    reshape_frames = np.expand_dims(reshape_frames, axis=0)
    del intervals
    del frames
    gc.collect(generation=2)
    return reshape_frames

# Segment Utils

In [19]:
def transform_into_segments(features, n_segments=32):
    """This will convert the vector 
    into 32 segments of feature vectors"""
    if features.shape[0] < n_segments:
        raise RuntimeError(
            "Number of prev segments lesser than expected output size"
        )

    cuts = np.linspace(0, features.shape[0], n_segments,
                       dtype=int, endpoint=False)

    new_feats = []
    for i, j in zip(cuts[:-1], cuts[1:]):
        new_feats.append(np.mean(features[i:j,:], axis=0))

    new_feats.append(np.mean(features[cuts[-1]:,:], axis=0))

    new_feats = np.array(new_feats)
    new_feats = sklearn.preprocessing.normalize(new_feats, axis=1)
    del cuts
    gc.collect(generation=2)
    return new_feats

# C3d

In [20]:
def C3D(weights='sports1M'):
    """Creation of the full C3D architecture
    :param weights: Weights to be loaded into the network. If None,
    the network is randomly initialized.
    :returns: Network model
    :rtype: keras.model
    """

    if weights not in {'sports1M', None}:
        raise ValueError('weights should be either be sports1M or None')

    if K.image_data_format() == 'channels_last':
        shape = (16, 112, 112, 3)
    else:
        shape = (3, 16, 112, 112)

    model = Sequential()
    model.add(
        Conv3D(64,
               3,
               activation='relu',
               padding='same',
               name='conv1',
               input_shape=shape))
    model.add(
        MaxPooling3D(pool_size=(1, 2, 2),
                     strides=(1, 2, 2),
                     padding='same',
                     name='pool1'))

    model.add(Conv3D(128, 3, activation='relu', padding='same', name='conv2'))
    model.add(
        MaxPooling3D(pool_size=(2, 2, 2),
                     strides=(2, 2, 2),
                     padding='valid',
                     name='pool2'))

    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3a'))
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3b'))
    model.add(
        MaxPooling3D(pool_size=(2, 2, 2),
                     strides=(2, 2, 2),
                     padding='valid',
                     name='pool3'))

    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4b'))
    model.add(
        MaxPooling3D(pool_size=(2, 2, 2),
                     strides=(2, 2, 2),
                     padding='valid',
                     name='pool4'))

    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5b'))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(
        MaxPooling3D(pool_size=(2, 2, 2),
                     strides=(2, 2, 2),
                     padding='valid',
                     name='pool5'))

    model.add(Flatten())

    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Dense(487, activation='softmax', name='fc8'))

    if weights == 'sports1M':
        model.load_weights(c3d_model_weights)

    return model

In [21]:
def c3d_feature_extractor():
    """Creation of the feature extraction architecture. This network is
    formed by a subset of the original C3D architecture (from the
    beginning to fc6 layer)
    :returns: Feature extraction model
    :rtype: keras.model
    """
    model = C3D()
    layer_name = 'fc6'
    feature_extractor_model = Model(inputs=model.input,
                                    outputs=model.get_layer(layer_name).output)
    return feature_extractor_model

In [22]:
feature_extractor = c3d_feature_extractor()

# Extract Features

## Normal Videos

In [28]:
normal_videos = os.listdir(normal_videos_path)
normal_videos.sort()
chunk = list(divide_chunks(normal_videos, 5))

### C3d

In [ ]:
print("Processing normal videos in chunks...")
for vid_name1 in chunk:
    for vid_name in vid_name1:
        print("Processing {}".format(vid_name))
        vid_path = os.path.join(normal_videos_path, vid_name)
        feats_path = os.path.join(
            raw_normal_train_features_c3d, vid_name[:-9] + ".npy")

        clips, frames = get_video_clips(vid_path, c3d_frame_count)
        print(frames)
        if frames > 8000:
            continue
        # Remove last clip if number of frames is not equal to 16
        if frames % 16 != 0:
            clips = clips[:-1]

        prep_clips = [preprocess_input_c3d(np.array(clip)) for clip in clips]
        prep_clips = np.vstack(prep_clips)

        features = feature_extractor.predict(prep_clips)
        del prep_clips
        features = sklearn.preprocessing.normalize(features, axis=1)   
        with open(feats_path, "wb") as f:
            np.save(f, features)
        del features
        gc.collect(generation=2)

Processing normal videos in chunks...
Processing Normal_Videos548_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


84225
Processing Normal_Videos549_x264.mp4
9264
Processing Normal_Videos550_x264.mp4
8484
Processing Normal_Videos551_x264.mp4
54000
Processing Normal_Videos552_x264.mp4
4283
Processing Normal_Videos553_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


12126
Processing Normal_Videos554_x264.mp4
3200
Processing Normal_Videos555_x264.mp4


...

897
Processing Normal_Videos629_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


4486
Processing Normal_Videos630_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


63384
Processing Normal_Videos631_x264.mp4
6790


In [29]:
raw_normal_train_features_c3d_list=os.listdir(raw_normal_train_features_c3d)
for filename in os.listdir(raw_normal_train_features_c3d):
    print("Processing {}".format(filename))
    raw_file_path = os.path.join(
        raw_normal_train_features_c3d, filename
    )
    processed_file_path = os.path.join(
        processed_normal_train_features_c3d, filename
    )
    with open(raw_file_path, "rb") as f:
        feats = np.load(f, allow_pickle=True)

    try:
        new_feats = transform_into_segments(feats)
        with open(processed_file_path, "wb") as f:
            np.save(f, new_feats, allow_pickle=True)
    except RuntimeError:
        print("Video {} too short".format(filename))

Processing Normal_Videos004.npy
Processing Normal_Videos007.npy
Processing Normal_Videos009.npy
Processing Normal_Videos011.npy
Processing Normal_Videos012.npy
Processing Normal_Videos013.npy
Processing Normal_Videos016.npy
Processing Normal_Videos020.npy
Video Normal_Videos020.npy too short
Processing Normal_Videos029.npy
Processing Normal_Videos038.npy
Processing Normal_Videos039.npy
Processing Normal_Videos040.npy
Processing Normal_Videos049.npy
Processing Normal_Videos053.npy
Video Normal_Videos053.npy too short
Processing Normal_Videos054.npy
Processing Normal_Videos055.npy
Video Normal_Videos055.npy too short
Processing Normal_Videos060.npy
Processing Normal_Videos062.npy
Processing Normal_Videos068.npy
Processing Normal_Videos071.npy
Processing Normal_Videos072.npy
Processing Normal_Videos074.npy
Processing Normal_Videos075.npy
Processing Normal_Videos078.npy
Processing Normal_Videos080.npy
Processing Normal_Videos088.npy
Processing Normal_Videos093.npy
Processing Normal_Videos0

## Anomaly Videos

In [43]:
abnormal_videos = os.listdir(abnormal_videos_path)
abnormal_videos.sort()
chunk = list(divide_chunks(abnormal_videos, 5))

### C3d

In [44]:
print("Processing abnormal videos in chunks...")
for vid_name1 in chunk:
    for vid_name in vid_name1:
        print("Processing {}".format(vid_name))
        vid_path = os.path.join(abnormal_videos_path, vid_name)
        feats_path = os.path.join(
            raw_abnormal_train_features_c3d, vid_name[:-9] + ".npy")

        clips, frames = get_video_clips(vid_path, c3d_frame_count)
        print(frames)
        if frames > 8000:
            continue
        # Remove last clip if number of frames is not equal to 16
        if frames % 16 != 0:
            clips = clips[:-1]

        prep_clips = [preprocess_input_c3d(np.array(clip)) for clip in clips]
        prep_clips = np.vstack(prep_clips)

        features = feature_extractor.predict(prep_clips)
        del prep_clips
        features = sklearn.preprocessing.normalize(features, axis=1)   
        with open(feats_path, "wb") as f:
            np.save(f, features)
        del features
        gc.collect(generation=2)

Processing abnormal videos in chunks...
Processing Vandalism001_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


1434
Processing Vandalism002_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


1841
Processing Vandalism003_x264.mp4


...

7183
Processing Vandalism049_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


7999
Processing Vandalism050_x264.mp4


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


899


In [45]:
raw_abnormal_train_features_c3d_list=os.listdir(raw_abnormal_train_features_c3d)
for filename in os.listdir(raw_abnormal_train_features_c3d):
    print("Processing {}".format(filename))
    raw_file_path = os.path.join(
        raw_abnormal_train_features_c3d, filename
    )
    processed_file_path = os.path.join(
        processed_abnormal_train_features_c3d, filename
    )
    with open(raw_file_path, "rb") as f:
        feats = np.load(f, allow_pickle=True)

    try:
        new_feats = transform_into_segments(feats)
        with open(processed_file_path, "wb") as f:
            np.save(f, new_feats, allow_pickle=True)
    except RuntimeError:
        print("Video {} too short".format(filename))

Processing Abuse001.npy
Processing Abuse002.npy
Processing Abuse003.npy
Processing Abuse005.npy
Processing Abuse006.npy
Processing Abuse007.npy
Processing Abuse009.npy
Processing Abuse010.npy
Processing Abuse011.npy
Processing Abuse012.npy
Processing Abuse013.npy
Processing Abuse014.npy
Processing Abuse015.npy
Processing Abuse016.npy
Processing Abuse017.npy
Processing Abuse018.npy
Processing Abuse019.npy
Processing Abuse020.npy
Processing Abuse021.npy
Processing Abuse022.npy
Processing Abuse023.npy
Processing Abuse024.npy
Processing Abuse025.npy
Processing Abuse026.npy
Processing Abuse027.npy
Processing Abuse028.npy
Processing Abuse029.npy
Video Abuse029.npy too short
Processing Abuse030.npy
Processing Abuse031.npy
Processing Abuse032.npy
Processing Abuse033.npy
Processing Abuse034.npy
Processing Abuse035.npy
Processing Abuse036.npy
Processing Abuse037.npy
Processing Abuse038.npy
Processing Abuse040.npy
Processing Abuse041.npy
Processing Abuse043.npy
Processing Abuse044.npy
Processing 